In [129]:
# Data wrangling
import pandas as pd
import numpy as np

# for Visulization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

#Dataset masuk
df2 = pd.read_csv('./laptops_test.csv')
df1 = pd.read_csv('./laptop_data.csv')

In [130]:
# Ganti nama fitur "Manufactur" -> "Company"
df2 = df2.rename(columns={'Manufacturer' : 'Company'})

# Ganti nama fitur "Category" -> "TypeName"
df2 = df2.rename(columns={'Category' : 'TypeName'})

# Ganti nama fitur CPU" -> "Cpu"
df2 = df2.rename(columns={"CPU": "Cpu"})

# Ganti nama fitur "RAM" -> "Ram"
df2 = df2.rename(columns={"RAM": "Ram"})

# Ganti nama fitur "Storage" -> "Memory"
df2 = df2.rename(columns={"Storage": "Memory"})

# Ganti nama fitur "GPU" -> "Gpu "
df2 = df2.rename(columns={"GPU": "Gpu "})

# Ganti nama fitur "Storage" -> "Memory"
df1 = df1.rename(columns={"Memory": "Storage"})

In [131]:
#Gabungin Buat Identitas Laptop
df1['Nama'] = df1.apply(lambda row: row['Company'] + " " + row['TypeName'] + " " + row['Cpu']+ " " + row['Ram'] , axis = 1)

#Gabungin Buat Identitas Laptop
df2['Nama'] = df2.apply(lambda row: row['Company'] + " " + row['TypeName'] + " " + row['Cpu']+ " " + row['Ram'] , axis = 1)
df2['Price'] = df2['Price'] / 100 

In [132]:
df0 = pd.concat([df1, df2], ignore_index=False)

In [133]:
df0 = df0.drop(["Inches", "Unnamed: 0", "Screen Size", "Screen", "Model Name", "ScreenResolution", "Weight", "Operating System", "Operating System Version"], axis = 1)

In [134]:
# Removing the GB string form Ram column
df0['Ram'] = df0['Ram'].str.replace('GB', '')
# Changing the dtype of column - Ram and Weight Column
df0['Ram'] = df0['Ram'].astype('int32')

In [135]:
# Memisahkan fitur storage menjadi daftar substring
split_prosesor = df0["Cpu"].str.split(" ")

# Mengambil angka frekuensi CPU
frekuensi_cpu = split_prosesor.str[-1]

# Mengkonversi frekuensi CPU ke float
df0["Cpu"] = frekuensi_cpu.str.replace("GHz", "")
df0["Cpu"] = df0['Cpu'].astype('float32')

In [136]:
# Replace type values
# berdasarkan persebaran dan rata-rata yang sudah ditampilkan
df0["TypeName"] = df0["TypeName"].replace({
    "2 in 1 Convertible": 3,
    "Gaming": 6,
    "Netbook": 1,
    "Notebook": 2,
    "Ultrabook": 4,
    "Workstation" : 5
})

In [137]:
#Karena Nama Berpengaruh pada program
# Hapus data duplikat berdasarkan kolom "nama"
df0 = df0.drop_duplicates()

In [138]:
#Price Process
#INR to IDR
df0['Price'] = df0['Price'] * 188.31 

#Redominasi
df0['Price'] = df0['Price'] / 1000000

#Penurunan harga
df0['Price'] = df0['Price'] - (25/100 * df0['Price'])

In [139]:
#Input Prediktor
X = df0[['TypeName', 'Price', 'Ram']]
y = df0['Cpu']

In [140]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1)

knn.fit(X, y)

# Predict the labels for the test data
y_pred = knn.predict(X)
df0['Cpu'] = y_pred

# Calculate the accuracy score
mse = np.mean((y - y_pred)**2)
print("Evaluasi Cpu : ", mse)

Evaluasi Cpu :  0.03798289


In [141]:
from sklearn.neighbors import KNeighborsClassifier

def proses(tipe, price) :
    
    encoder = {
        "2 in 1 Convertible": 3,
        "Gaming": 6,
        "Netbook": 1,
        "Notebook": 2,
        "Ultrabook": 4,
        "Workstation" : 5
        }
    
    laptop = ''
    x1 = [[encoder[tipe], price]]
    
    X = df0[['TypeName', 'Price']]
    y = df0['Nama']
    
    knn2 = KNeighborsClassifier(n_neighbors=1)
    knn2.fit(X, y)

    # Predict the labels for the test data
    laptop = knn2.predict(x1)
    
    print(laptop)
    
print("Halo!! ini input pencarian laptop untuk pemula\n")
tipe = input("Masukkan Tipe Laptop: ")
price = int(input("Masukkan Budget: "))

proses(tipe, price)

Halo!! ini input pencarian laptop untuk pemula

Masukkan Tipe Laptop: Ultrabook
Masukkan Budget: 20
['Toshiba Ultrabook Intel Core i5 6200U 2.3GHz 8GB']


C:\Users\ragil\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [142]:
def proses2(tipe, price, ram, cpu) :
    
    encoder = {
        "2 in 1 Convertible": 3,
        "Gaming": 6,
        "Netbook": 1,
        "Notebook": 2,
        "Ultrabook": 4,
        "Workstation" : 5
        }
    
    laptop1 = ''
    x1 = [[encoder[tipe], price, ram, cpu]]
    
    X = df0[['TypeName', 'Price','Ram', 'Cpu']]
    y = df0['Nama']
    
    knn3 = KNeighborsClassifier(n_neighbors=1)
    knn3.fit(X, y)

    # Predict the labels for the test data
    laptop1 = knn3.predict(x1)
    
    print(laptop1)
    
print("Halo!! ini input pencarian laptop\n")
tipe = input("Masukkan Tipe Laptop : ")
price = int(input("Masukkan Budget (Jt): "))
ram = int(input("Masukkan RAM (GB): "))
cpu = float(input("Masukkan CPU (Ghz): "))

proses2(tipe, price, ram, cpu)

Halo!! ini input pencarian laptop

Masukkan Tipe Laptop : Ultrabook
Masukkan Budget (Jt): 20
Masukkan RAM (GB): 32
Masukkan CPU (Ghz): 3
['Toshiba Ultrabook Intel Core i7 7500U 2.7GHz 32GB']


C:\Users\ragil\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
